In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
import chromadb
import pandas as pd

In [4]:
def read_vectordb_as_df(db_path:str):
    client = chromadb.PersistentClient(path=db_path)
    for collection in client.list_collections():
        data = collection.get(include=['embeddings', 'documents', 'metadatas'])
        df = pd.DataFrame({"ids":data["ids"], 
                        #    "embeddings":data["embeddings"], 
                            "metadatas":data["metadatas"], 
                            "documents":data["documents"]})
        df["first_div"] = df["metadatas"].apply(lambda x: x["First Division"])
        df["second_div"] = df["metadatas"].apply(lambda x: x["Second Division"])
        df["filename"] = df["metadatas"].apply(lambda x: x["File Name"])
        df = df[["ids", "first_div", "second_div","filename","documents", "metadatas"]]
    return df

def get_first_div(db_path:str):
    df = read_vectordb_as_df(db_path=db_path)
    docs_list = df["first_div"].unique().tolist()
    docs_list.sort()
    return docs_list

def get_second_div(db_path:str):
    df = read_vectordb_as_df(db_path=db_path)
    docs_list = df["second_div"].unique().tolist()
    docs_list.sort()
    return docs_list

In [6]:
db_path = "./db/chroma_db_02"
rag_target1 = get_first_div(db_path=db_path)
rag_target2 = get_second_div(db_path=db_path)
rag_target = rag_target1 + rag_target2
rag_target

['MANUAL',
 'PORT',
 'Rules',
 'ABS',
 'BV',
 'Common',
 'Cryostar',
 'DNV',
 'Integrated Smart Ship(ISS)',
 'KR',
 'MARPOL',
 'Port Regulation',
 'SOLAS',
 'Win GD']

In [7]:
rag_target.insert(0, "vectorstore")
rag_target.insert(0, "vectordb")
docs = ", ".join(rag_target)
docs

'vectordb, vectorstore, MANUAL, PORT, Rules, ABS, BV, Common, Cryostar, DNV, Integrated Smart Ship(ISS), KR, MARPOL, Port Regulation, SOLAS, Win GD'

In [8]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_ollama.chat_models import ChatOllama
from pydantic import BaseModel, Field

In [ ]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "web_search", "database"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore or a database.",
    )

In [ ]:
# Prompt
system = f"""You are an expert at routing a user question to a vectorstore, web search or database.
The vectorstore contains documents related to {docs}.
Use the vectorstore for questions on these topics. 
The question contains words related to database.
Use the database for questions on these topics. 
Otherwise, use web-search."""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

In [29]:
llm = ChatGroq(temperature=0, model_name= "llama-3.2-11b-text-preview")
# llm = ChatOllama(base_url="http://localhost:11434", model="llama3.2:latest")
structured_llm_router = llm.with_structured_output(RouteQuery)
question_router = route_prompt | structured_llm_router
question_router

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert at routing a user question to a vectorstore or web search or database.\nThe vectorstore contains documents related to vectordb, vectorstore, MANUAL, PORT, Rules, ABS, BV, Common, Cryostar, DNV, Integrated Smart Ship(ISS), KR, MARPOL, Port Regulation, SOLAS, Win GD.\nUse the vectorstore for questions on these topics. \nThe question contains words related to database.\nUse the database for questions on these topics. \nOtherwise, use web-search.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D0A6CE6C60>, async_client=

In [34]:
print(question_router.invoke({"question": "what is eda?"}))
print(question_router.invoke({"question": "according to vectorstore, what is eda?"}))
print(question_router.invoke({"question": "according to rule, what is eda?"}))  # 살짝 다르게
print(question_router.invoke({"question": "according to port regulation, what is eda?"})) 
print(question_router.invoke({"question": "according to menual, what is eda?"}))  # 일부러 오타
print(question_router.invoke({"question": "about win gd, what is eda?"})) 
print(question_router.invoke({"question": "with reference to Rules, summaize RESOLUTION MEPC.248(66)"}))
print(question_router.invoke({"question": "check the pressure in database"}))
print(question_router.invoke({"question": "who is Son Heung-min"}))
print(question_router.invoke({"question": "according to database, who is Son Heung-min"}))
print(question_router.invoke({"question": "according to vectordb, who is Son Heung-min"}))

datasource='web_search'
datasource='vectorstore'
datasource='vectorstore'
datasource='vectorstore'
datasource='vectorstore'
datasource='vectorstore'
datasource='vectorstore'
datasource='database'
datasource='web_search'
datasource='database'
datasource='vectorstore'
